# Youtube trends 
https://www.kaggle.com/datasnaek/youtube-new

In [ ]:
country_codes = ['CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']


In [ ]:
import pandas as pd
from pandas import DataFrame
import json
import csv

In [ ]:
categories = {}
data = {}

for country_code in country_codes:
    # load videos per country
    print(country_code)


    category_fn = f'raw/{country_code}_category_id.json'

    with open(category_fn, 'r') as f:
        cats = json.loads(f.read())
        categories[country_code] = {entry['id'] : entry['snippet']['title'] for entry in cats['items']}

    
    video_fn = f'raw/{country_code}videos.csv'

    try:
        data[country_code]: DataFrame = pd.read_csv(video_fn, encoding='utf-8') 
    except Exception as e:
        print(f"Error for {country_code} : {e}")


## Prepare categories

In [ ]:
# create one large map for all global categories as they are just 
global_cats = {}

for _, cats in categories.items():
    for cat in cats.items():
        if cat[0] not in global_cats: 
            global_cats[cat[0]] = []

        global_cats[cat[0]].append(cat[1])

assert len({key : len(set(val)) for key, val in global_cats.items() if len(set(val)) > 1}) == 0, 'NOT unique and same names for categories for all countries'

global_categories = {key : val[0] for key, val in global_cats.items()}

In [ ]:
with open('categories.json', 'w') as file:
    file.write(json.dumps(global_categories))

In [ ]:
global_categories

## Prepare data

In [ ]:
# add country info and category names
for i, cc in enumerate(country_codes):
    data[cc]['country_code'] = cc
    data[cc]['country_id'] = i

    data[cc]['category_name'] = data[cc].apply(lambda row: global_categories[str(row['category_id'])], axis=1)

In [ ]:
data['JP'].tail(1)

In [ ]:
sum_ = 0
for cc in country_codes:
    columns = data[cc].columns
    print(f"{cc} ({len(columns)}) ({data[cc].size}): {columns}")
    sum_ += len(data[cc])

print(sum_)

In [ ]:
dfs = pd.concat(list(data.values()), ignore_index=True, join='inner')
dfs

In [ ]:
dfs.to_csv('videos.csv')